In [1]:
import pandas as pd
import numpy as np
import filtering

데이터 불러오기

In [2]:
raw_rating_data= pd.read_csv('./data/rating.csv')

In [3]:
raw_movie_data= pd.read_csv('./data/movie.csv')

필터링

In [4]:
# 필터링 
rating_threshold = 1000
genres_included = ['Action', 'Crime']
movie_min_year = 1980

In [5]:
#threshold 100 -> 15970204개 행 /// 1000 -> 2801066개 행
rating_data = filtering.rating_filter(raw_rating_data, rating_threshold) 

In [6]:
movie_data = filtering.movie_filter(raw_movie_data, movie_min_year, genres_included)

은선님 코드

In [7]:
rating_datas= pd.merge(rating_data, movie_data, on = 'movieId', how = 'right')

In [8]:
from scipy.sparse.linalg import svds
def make_prediction_df():
    R_df = rating_datas.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)

    #R은 pivot_table을 numpy matrix로 만든 것
    R = R_df.values
    
    #user_rating_mean은 사용자의 평균 평점
    user_ratings_mean = np.mean(R, axis = 1)
    
    #R_demeaned : 사용자-영화 테이블에 대해 사용자 평균 평점을 뺀 것
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
        
    #U 행렬, sigma 행렬, V전치행렬을 반환.
    #이때 spicy에 있는 svd를 이용한다.
    U, sigma, Vt = svds(R_demeaned, k = 50)
    
    #sigma는 0이 포함되지 않은 값으로만 구성되어 있다.
    #sigma를 0이 포함된 대칭 행렬로 변환한다.
    sigma = np.diag(sigma)
    
    #SVD가 적용되어 분해된 R_demeaned를 원본 행렬로 복구
    #구한 원본 행렬에 사용자 평균 rating을 더해준다.
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1,1)
    
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
    
    return preds_df


In [9]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    #인덱스로 변환해주어야 해서 -1 
    user_row_number = userID - 1
    
    #앞에서 만든 prediction행렬을 사용자 인덱스에 따라 영화 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    #원본 rating_data에서 user_id에 해당하는 데이터를 뽑아낸다.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    
    #user_data와 원본 영화 데이터를 합친다.
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )
    
    #원본 영화 데이터에서 사용자가 본 영화를 제외한 데이터 추출
    #.merge로 sorted_user_predcitions와 user_full데이터를 합친다.
    #.rename으로 컬럼 이름을 바꾸고 정렬한다
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False)
                      )
                       
    return recommendations


In [10]:
#make_prediction_df()함수 호출하기 전에 input을 넣어줍니다.
preds_df = make_prediction_df()

In [11]:
#24번 사용자가 본 영화 데이터를 토대로 영화 추천 받은 목록
prediction= recommend_movies(preds_df, 24, movie_data, rating_data, 10)

In [12]:
prediction.head(10)

,movieId,title,genres,year,Predictions
28,110,Braveheart (1995),"[Action, Drama, War]",1995.0,5.888322
70,318,"Shawshank Redemption, The (1994)","[Crime, Drama]",1994.0,5.673850
220,1215,Army of Darkness (1993),"[Action, Adventure, Comedy, Fantasy, Horror]",1993.0,5.259624
1206,7153,"Lord of the Rings: The Return of the King, The...","[Action, Adventure, Drama, Fantasy]",2003.0,5.148396
374,2005,"Goonies, The (1985)","[Action, Adventure, Children, Comedy, Fantasy]",1985.0,5.136931
215,1197,"Princess Bride, The (1987)","[Action, Adventure, Comedy, Fantasy, Romance]",1987.0,5.065103
128,553,Tombstone (1993),"[Action, Drama, Western]",1993.0,5.048535
214,1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Sci-Fi]",1980.0,4.967289
65,293,Léon: The Professional (a.k.a. The Professiona...,"[Action, Crime, Drama, Thriller]",1994.0,4.964584
379,2028,Saving Private Ryan (1998),"[Action, Drama, War]",1998.0,4.935194
